In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

import com.toldoven.aoc.notebook.AocClient

val aoc = AocClient.fromFile().interactiveDay(2024, 9)

In [2]:
val input = aoc.input().toCharArray().map { it.digitToInt() }

fun mapToBlocks(driveMap: List<Int>): List<Int?> {
    return buildList {
        driveMap.chunked(2).forEachIndexed{ fileId, chunk ->
            val fileBlocks = chunk[0]
            val spaceBlocks = if (chunk.size == 2) chunk[1] else 0
            repeat(fileBlocks) {
                add(fileId)
            }
            repeat(spaceBlocks) {
                add(null)
            }
        }
    }
}

fun checksum(blocks: List<Int?>): Long {
    return blocks
        .mapIndexed { index, fileId ->
            fileId?.let { (index * fileId).toLong() } ?: 0
        }.sum()
}

fun compact(blocks: List<Int?>): List<Int?> {
    val _blocks = blocks.toMutableList()
    var end = _blocks.lastIndex

    for (index in _blocks.indices) {
        if (index < end && _blocks[index] == null) {
            val b = _blocks.indexOfLast { it != null }
            _blocks[index] = _blocks[b]
            _blocks[b] = null
            end = b
        }
    }

    return _blocks.toList()
}

val blocks = mapToBlocks(input)
val checksum = checksum(compact(blocks))
aoc.submitPartOne(checksum)

The answer is correct! Your answer: 6331212425418 You already completed this part.

In [7]:
data class FileDescriptor(val fileId: Int, val index: Int, val size: Int)

fun mapToLayout(driveMap: List<Int>): Pair<List<FileDescriptor>, MutableList<Pair<Int, Int>>> {
    val files = mutableListOf<FileDescriptor>() // fileId, index, size
    val spaces = mutableListOf<Pair<Int, Int>>() // index of first block, size
    var index = 0

    driveMap.chunked(2).forEachIndexed { fileId, chunk ->
        files.add(FileDescriptor(fileId, index, chunk[0]))
        index += chunk[0]
        if (chunk.size == 2) {
            spaces.add(index to chunk[1])
            index += chunk[1]
        }
    }
    return Pair(files, spaces)
}

fun compactWhole(blocks: List<Int?>, driveMap: List<Int>): List<Int?> {
    val _blocks = blocks.toMutableList()

    val (files, spaces) = mapToLayout(driveMap)

    fun move(file: FileDescriptor, spaceIndex: Int) {
        val (newIndex, spaceSize) = spaces[spaceIndex]
        for (i in newIndex..<newIndex+file.size) {
            _blocks[i] = file.fileId
        }
        for (i in file.index..<file.index+file.size) {
            _blocks[i] = null
        }
        if (file.size < spaceSize) {
            spaces[spaceIndex] = newIndex+file.size to spaceSize - file.size
        } else {
            spaces.removeAt(spaceIndex)
        }
    }

    for (file in files.reversed()) {
        val newSpaceIndex = spaces.indexOfFirst { it.second >= file.size && it.first < file.index }
        if (newSpaceIndex >= 0) {
            move(file, newSpaceIndex)
        }
    }

    return _blocks.toList()
}
val example = "2333133121414131402".toCharArray().map { it.digitToInt() }
compactWhole(mapToBlocks(example), example)

[0, 0, 9, 9, 2, 1, 1, 1, 7, 7, 7, null, 4, 4, null, 3, 3, 3, null, null, null, null, 5, 5, 5, 5, null, 6, 6, 6, 6, null, null, null, null, null, 8, 8, 8, 8, null, null]

In [8]:
val check2 = checksum(compactWhole(blocks, input))
check2

6363268339304

In [9]:
aoc.submitPartTwo(check2)

Your answer: 6363268339304. 
 That's the right answer! You are one gold star closer to finding the Chief Historian. 
 You have completed Day 9! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .